<a href="https://colab.research.google.com/github/pavi-ninjaac/DeepLearningProjects/blob/main/Face_Emotion_Recognision/FER_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#keras 
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

#keras layers
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,AveragePooling2D
from keras.layers import Flatten,Dropout,Dense

In [4]:
#read the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataFiles/fer2013.csv')

print("the classs labels",data['emotion'].unique()) #7 classes
print("The shape of the dataset",data.shape)

data.head(4)

the classs labels [0 2 4 6 3 5 1]
The shape of the dataset (35887, 3)


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training


In [5]:
data['Usage'].value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [6]:
#pixcels are in str formate so need to read and convert it to numpy array

def generate_dataset(data):
  X_train,y_train,X_test,y_test,X_val,y_val = [],[],[],[],[],[]

  print("Collecting all data...................")
  for i in range(data.shape[0]):
    d = data.iloc[i,:]
    value = d['pixels'].split(' ')
    if (d['Usage'] == 'Training'):
      X_train.append(np.array(value,'float32'))
      y_train.append(d['emotion'])
    elif d['Usage'] =='PrivateTest':
      X_val.append(np.array(value,'float32'))
      y_val.append(d['emotion'])
    else:
      X_test.append(np.array(value,'float32'))
      y_test.append(d['emotion'])


  print("Converting to numpy array>>>>>>>>>>>>>>>>>>")
  #convert list to numpy array
  X_train = np.array(X_train,'float32')  
  y_train = np.array(y_train,'float32')  
  X_test = np.array(X_test,'float32')  
  y_test = np.array(y_test,'float32')
  X_val = np.array(X_val,'float32')  
  y_val = np.array(y_val,'float32')

  print("Normalizing the data>>>>>>>>>>>>>>>>>>>>>>")
  #normalize the data
  X_train  = X_train/255.0
  X_test = X_test/255.0
  X_val = X_val/255.0

  print("Converting target to one hot encoded values>>>>>>>>>>>>>>>>>>>>>>")
  #convert to numerical values to 0,1
  y_train = to_categorical(y_train,num_classes=7)
  y_test = to_categorical(y_test,num_classes=7)
  y_val = to_categorical(y_val,num_classes=7)

  print("reshaping the data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
  #reshape the train X data to 48 x 48 picxels
  #the len of the given picxels is 2304 ------>(48*48)
  X_train = X_train.reshape(X_train.shape[0] , 48 ,48 , 1)
  X_test = X_test.reshape(X_test.shape[0] , 48 ,48 , 1)
  X_val = X_val.reshape(X_val.shape[0] , 48 ,48 , 1)

  print("Preprocessing  completed!!!!!!!!!! stay happy :)")
  return X_train,y_train,X_test,y_test,X_val,y_val

In [7]:
X_train,y_train,X_test,y_test,X_val,y_val = generate_dataset(data)

Converting to numpy array>>>>>>>>>>>>>>>>>>
Normalizing the data>>>>>>>>>>>>>>>>>>>>>>
Converting target to one hot encoded values>>>>>>>>>>>>>>>>>>>>>>
reshaping the data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Preprocessing  completed!!!!!!!!!! stay happy :)


In [8]:
print("The size of the train data-------------------->",X_train.shape)
print("The size of the train target data------------->",y_train.shape)
print()
print("The size of the test data--------------------->",X_test.shape)
print("The size of the test target data-------------->",y_test.shape)
print()
print("The size of the validation data--------------->",X_val.shape)
print("The size of the validation target data-------->",y_val.shape)

The size of the train data--------------------> (28709, 48, 48, 1)
The size of the train target data-------------> (28709, 7)

The size of the test data---------------------> (3589, 48, 48, 1)
The size of the test target data--------------> (3589, 7)

The size of the validation data---------------> (3589, 48, 48, 1)
The size of the validation target data--------> (3589, 7)


# building the model--->ALEXNET Architechture

In [31]:
#building the model
def alexnet(input_shape , num_classes , ):
  model = Sequential()
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same', input_shape=input_shape)) 
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same', ))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))

  #model.add(Conv2D(64, (5, 5), activation='relu', padding='same', ))
  #model.add(BatchNormalization())
  #model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Conv2D(128, (3, 3), activation='relu',  padding='same', ))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same',))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Conv2D(256, (3, 3), activation='relu', padding='same',))
  model.add(Conv2D(256, (3, 3), activation='relu',  padding='same',))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Flatten())

  model.add(Dense(1024, activation='relu',))
  model.add(Dropout(0.2))
  model.add(Dense(1024, activation='relu', ))
  model.add(Dropout(0.2))
  model.add(Dense(num_classes, activation='sigmoid'))


  # compile model
  
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  #model summary
  print(model.summary())

  return model

In [32]:
model = alexnet(input_shape = (48,48,1), num_classes = 7)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_17 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 24, 24, 128)       147584    
_________________________________________________________________
batch_normalization_18 (Batc (None, 24, 24, 128)      

In [ ]:
# set callbacks
early_stoppping = EarlyStopping(monitor = 'val_loss',
                                min_delta = 0.001,
                                patience = 10,
                                restore_best_weights=True)

#set the global values
epoches = 70
batch_size = 64

#fit the model
history = model.fit(X_train, y_train,  
          batch_size=batch_size,  
          epochs=epoches,  
          verbose=1,  
          validation_data=(X_test, y_test),  
          shuffle=True)

Epoch 1/70
449/449 [==============================] - 17s 37ms/step - loss: 2.4312 - accuracy: 0.2399 - val_loss: 1.7029 - val_accuracy: 0.3274
Epoch 2/70
449/449 [==============================] - 16s 36ms/step - loss: 1.6219 - accuracy: 0.3572 - val_loss: 1.5705 - val_accuracy: 0.3661
Epoch 3/70
449/449 [==============================] - 16s 36ms/step - loss: 1.4781 - accuracy: 0.4177 - val_loss: 1.4450 - val_accuracy: 0.4528
Epoch 4/70
449/449 [==============================] - 16s 36ms/step - loss: 1.3288 - accuracy: 0.4831 - val_loss: 1.2828 - val_accuracy: 0.5116
Epoch 5/70
449/449 [==============================] - 16s 36ms/step - loss: 1.2333 - accuracy: 0.5279 - val_loss: 1.2932 - val_accuracy: 0.5171
Epoch 6/70
449/449 [==============================] - 16s 36ms/step - loss: 1.1321 - accuracy: 0.5752 - val_loss: 1.2938 - val_accuracy: 0.4990
Epoch 7/70
449/449 [==============================] - 16s 36ms/step - loss: 1.0497 - accuracy: 0.6023 - val_loss: 1.2176 - val_accuracy: